In [34]:
# %%capture
# !pip install dlt[duckdb] 

import dlt
import duckdb

In [35]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [36]:
sum_generator = 1.0 + 1.4142135623730951 + 1.7320508075688772 + 2.0 + 2.23606797749979
sum_generator

8.382332347441762

In [37]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


In [54]:
import duckdb
import dlt

square_root_generator_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}
    

def people_2():
    for i in range(3, 9):  # Starting from 6 to avoid overlapping IDs
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}
        
        
table_one = square_root_generator_pipeline.run(people_1(),table_name="table_one", write_disposition="replace")
print(table_one)        

table_two = square_root_generator_pipeline.run(people_2(),table_name="table_two", write_disposition="replace")
print(table_two)        

Pipeline dlt_ipykernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///C:\Users\Anwender\Jose Portilla DAnalysis\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707333304.5552182 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///C:\Users\Anwender\Jose Portilla DAnalysis\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707333305.5524275 is LOADED and contains no failed jobs


In [53]:
import duckdb

# Define generator functions for people
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Create a DuckDB connection
conn = duckdb.connect(":memory:")

# Create a new table with a primary key on ID
conn.execute("""
CREATE TABLE people (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
""")

# Load data from the first generator into the table
for person in people_1():
    conn.execute("INSERT INTO people VALUES (?, ?, ?, ?, NULL)", 
                 (person["ID"], 
                  person["Name"], 
                  person["Age"], 
                  person["City"]))

# Merge data from the second generator into the same table
for person in people_2():
    conn.execute("""
    INSERT INTO people VALUES (?, ?, ?, ?, ?)
    ON CONFLICT(ID) DO UPDATE SET Name=excluded.Name, 
    Age=excluded.Age, 
    City=excluded.City, 
    Occupation=excluded.Occupation
    """, 
        (person["ID"], 
          person["Name"], 
          person["Age"], 
          person["City"], 
          person["Occupation"]))

# Fetch the combined data from the 'people' table
combined_data = conn.execute("SELECT * FROM people").fetch_df()

# Display the combined data
print(combined_data,'\n\n')

# Calculate the sum of ages of all people in the 'people' table
sum_of_ages = conn.execute("SELECT SUM(Age) FROM people").fetchone()[0]

# Close the connection
conn.close()

# Print the sum of ages
print("Sum of all ages of people:", sum_of_ages)


   ID      Name  Age    City Occupation
0   1  Person_1   26  City_A       None
1   2  Person_2   27  City_A       None
2   3  Person_3   33  City_B      Job_3
3   4  Person_4   34  City_B      Job_4
4   5  Person_5   35  City_B      Job_5
5   6  Person_6   36  City_B      Job_6
6   7  Person_7   37  City_B      Job_7
7   8  Person_8   38  City_B      Job_8 


Sum of all ages of people: 266


# Question Nr 3 - Sum of all ages

In [59]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Sum of ages for people_1
total_age_1 = sum(person["Age"] for person in people_1())

# Sum of ages for people_2
total_age_2 = sum(person["Age"] for person in people_2())

# Combine the totals
total_age_combined = total_age_1 + total_age_2

print("Total Age:", total_age_combined)


Total Age: 353


## Question Nr 4 - Sum all ages with ID

In [60]:
import duckdb
import dlt

# Define generator functions for people
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Create a DuckDB connection
conn = duckdb.connect(":memory:")

# Create a new table with a primary key on ID
conn.execute("""
CREATE TABLE people (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
""")

# Load data from the first generator into the table
for person in people_1():
    conn.execute("INSERT INTO people VALUES (?, ?, ?, ?, NULL)", 
                 (person["ID"], 
                  person["Name"], 
                  person["Age"], 
                  person["City"]))

# Merge data from the second generator into the same table
for person in people_2():
    conn.execute("""
    INSERT INTO people VALUES (?, ?, ?, ?, ?)
    ON CONFLICT(ID) DO UPDATE SET Name=excluded.Name, 
    Age=excluded.Age, 
    City=excluded.City, 
    Occupation=excluded.Occupation
    """, 
        (person["ID"], 
          person["Name"], 
          person["Age"], 
          person["City"], 
          person["Occupation"]))


combined_data = conn.execute("SELECT * FROM people").fetch_df()
print(combined_data,'\n\n')

# Calculate the sum of ages of all people in the 'people' table
sum_of_ages = conn.execute("SELECT SUM(Age) FROM people").fetchone()[0]

# Close the connection
conn.close()

# Print the sum of ages
print("Sum of all ages of people:", sum_of_ages)


   ID      Name  Age    City Occupation
0   1  Person_1   26  City_A       None
1   2  Person_2   27  City_A       None
2   3  Person_3   33  City_B      Job_3
3   4  Person_4   34  City_B      Job_4
4   5  Person_5   35  City_B      Job_5
5   6  Person_6   36  City_B      Job_6
6   7  Person_7   37  City_B      Job_7
7   8  Person_8   38  City_B      Job_8 


Sum of all ages of people: 266
